### Import necessary libraries and packages 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geocoder
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# import BeautifulSoup for package for web scraping
!pip install beautifulsoup4
!pip install lxml
!pip install et_xmlfile

from bs4 import BeautifulSoup
import lxml
import requests

import os
from functools import reduce

print('Libraries imported.')

     |████████████████████████████████| 102kB 14.3MB/s ta 0:00:01
     |████████████████████████████████| 829kB 34.0MB/s eta 0:00:01
     |████████████████████████████████| 81kB 19.7MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_

### Request the content of the London boroughs from a wiki page and assign to a variable

In [3]:
os.system('wget -q -O "boroughs.html" "https://en.wikipedia.org/wiki/List_of_London_boroughs"')

with open('boroughs.html', encoding='utf-8-sig') as fp:
    soup = BeautifulSoup(fp,"lxml")

### Extract the table from the wiki page

In [4]:
table = soup.find('table',{'class':'wikitable sortable'})
# define the dataframe rows and columns
rows = table.find_all('tr')
columns = columns = [v.text.replace('\n','') for v in rows[0].find_all('th')] 
# instantiate the dataframe
df = pd.DataFrame(columns=columns)

for i in range(1,len(rows)):
    tds = rows[i].find_all('td')
    values = [td.text.replace('\n','').replace(u'\ufeff', '') for td in tds]
    df=df.append(pd.Series(values,index=columns),ignore_index=True)
    
df=df[['Borough','Co-ordinates']]
df['Borough'] = df['Borough'].str.replace(r"\[.*?\]","").str.strip()
df['Latitude'] = df['Co-ordinates'].str.extract('(\d\d.\d\d\d\d)',expand=True).astype(float)
df['Longitude'] = df['Co-ordinates'].str.split().str[7].astype(float)

df=df[['Borough','Latitude','Longitude']]
df

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198
5,Camden,51.5290,-0.1255
6,Croydon,51.3714,-0.0977
7,Ealing,51.5130,-0.3089
8,Enfield,51.6538,-0.0799
9,Greenwich,51.4892,0.0648


### Extract crime data from the internet and put into Dataframe

In [5]:
website_url1 = requests.get('https://www.finder.com/uk/london-crime-statistics').text

soup1 = BeautifulSoup(website_url1,'html.parser')
  
table1 = soup1.find('table',{'class':"luna-table js-tablesorter"})
rows1 = table1.find_all('tr')
columns1 = ['Borough','Crime count']
df1 = pd.DataFrame(columns=columns1)

for i in range(1,len(rows1)):
    tds1 = rows1[i].find_all('td')
    values1 = [td.text.replace('\n','') for td in tds1]
    df1=df1.append(pd.Series(values1,index=columns1),ignore_index=True)

df1.sort_values(by=['Borough'],ascending=True,inplace=True)
df1=df1.reset_index(drop=True)

df1['Crime count'] = df1['Crime count'].str.replace(',', '').astype(int)

### Extracting house prices by borough from the internet and put into Dataframe

In [6]:
website_url2 = requests.get('https://www.theweek.co.uk/99093/london-house-prices-which-boroughs-are-falling-and-which-are-on-the-rise').text

soup2 = BeautifulSoup(website_url2,'html.parser')

In [7]:
table2 = soup2.find('table')
rows2 = table2.find_all('tr')
columns2 = ['Borough','Average house price(£)','','']
df2 = pd.DataFrame(columns=columns2)

for i in range(2,len(rows2)):
    tds2 = rows2[i].find_all('td')
    values2 = [td.text for td in tds2]
    df2=df2.append(pd.Series(values2,index=columns2),ignore_index=True)

df2.sort_values(by=['Borough'],ascending=True,inplace=True)
df2=df2.reset_index(drop=True).drop(columns=['', ''])
df2['Average house price(£)']=df2['Average house price(£)'].str.replace('£','').str.replace(',','').astype(float)
df2.head()

,Borough,Average house price(£)
0,Barking and Dagenham,312010.0
1,Barnet,622784.0
2,Bexley,402823.0
3,Brent,557527.0
4,Bromley,533716.0


### Combine all the dataframes together

In [8]:
dfs = [df, df1, df2]
london_data = reduce(lambda left,right: pd.merge(left,right,on='Borough'), dfs)
london_data

,Borough,Latitude,Longitude,Crime count,Average house price(£)
0,Barking and Dagenham,51.5607,0.1557,19836,312010.0
1,Barnet,51.6252,-0.1517,30926,622784.0
2,Bexley,51.4549,0.1505,16932,402823.0
3,Brent,51.5588,-0.2817,33039,557527.0
4,Bromley,51.4039,0.0198,25125,533716.0
5,Camden,51.5290,-0.1255,38691,968276.0
6,Croydon,51.3714,-0.0977,33546,423341.0
7,Ealing,51.5130,-0.3089,30920,540036.0
8,Enfield,51.6538,-0.0799,30561,443516.0
9,Greenwich,51.4892,0.0648,27865,431318.0


### Get the geographical coordinates of London.

In [9]:
address = 'London, UK'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London, UK are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London, UK are 51.5073219, -0.1276474.


### Visualise London with the boroughs in it.

In [10]:
london_data.dtypes

Borough                    object
Latitude                  float64
Longitude                 float64
Crime count                 int64
Average house price(£)    float64
dtype: object

In [11]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(london_data['Latitude'], london_data['Longitude'], london_data['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

### Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'EVMH5NBPBWYKKUZJ00IIATXNF3IKSERRGYZ2ORFHESWRU5S4' # your Foursquare ID
CLIENT_SECRET = 'XU3XQVHFDOFKRPYS2OTS1SMWH1RKKKNFAOIDWDLGAS1WK20Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EVMH5NBPBWYKKUZJ00IIATXNF3IKSERRGYZ2ORFHESWRU5S4
CLIENT_SECRET:XU3XQVHFDOFKRPYS2OTS1SMWH1RKKKNFAOIDWDLGAS1WK20Y


#### Explore the first borough in our dataframe.

In [13]:
london_data.loc[0, 'Borough']

'Barking and Dagenham'

Get the borough's latitude and longitude values.

In [14]:
borough_latitude = london_data.loc[0, 'Latitude'] # neighborhood latitude value
borough_longitude = london_data.loc[0, 'Longitude'] # neighborhood longitude value

borough_name = london_data.loc[0, 'Borough'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(borough_name, 
                                                               borough_latitude, 
                                                               borough_longitude))

Latitude and longitude values of Barking and Dagenham are 51.5607, 0.1557.


#### Get the top 100 venues that are in Barking and Dagenham within a radius of 1.5 kilometers.
First, let's create the GET request URL. Name your URL **url**.

In [15]:
LIMIT = 100
radius = 1500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    borough_latitude, 
    borough_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

Borrow the **get_category_type** function from the Foursquare lab.

In [16]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean the json and structure it into a *pandas* dataframe.

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Central Park,Park,51.559560,0.161981
1,Lara Grill,Turkish Restaurant,51.562445,0.147178
2,Iceland,Grocery Store,51.560578,0.147685
3,Shell,Gas Station,51.560415,0.148364
4,Asda,Supermarket,51.565751,0.143392


## Explore Boroughs in London
#### Create a function to repeat the same process to all the boroughs in London

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each borough and create a new dataframe called *london_venues*.

In [19]:
london_venues = getNearbyVenues(names=london_data['Borough'],
                                   latitudes=london_data['Latitude'],
                                   longitudes=london_data['Longitude']
                                )

Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


#### Check the size of the resulting dataframe

In [20]:
print(london_venues.shape)
london_venues.head()

(2472, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,51.5607,0.1557,Central Park,51.559560,0.161981,Park
1,Barking and Dagenham,51.5607,0.1557,Lara Grill,51.562445,0.147178,Turkish Restaurant
2,Barking and Dagenham,51.5607,0.1557,Iceland,51.560578,0.147685,Grocery Store
3,Barking and Dagenham,51.5607,0.1557,Shell,51.560415,0.148364,Gas Station
4,Barking and Dagenham,51.5607,0.1557,Asda,51.565751,0.143392,Supermarket


Let's check how many venues were returned for each neighborhood

In [21]:
london_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Barking and Dagenham,18,18,18,18,18,18
Barnet,18,18,18,18,18,18
Bexley,57,57,57,57,57,57
Brent,100,100,100,100,100,100
Bromley,63,63,63,63,63,63
Camden,100,100,100,100,100,100
Croydon,90,90,90,90,90,90
Ealing,100,100,100,100,100,100
Enfield,73,73,73,73,73,73


#### Find out how many unique categories can be curated from all the returned venues

In [22]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 264 uniques categories.


## Analyze Each Borough

In [23]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add borough column back to dataframe
london_onehot['Borough'] = london_venues['Borough'] 

# move borough column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Borough,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Caribbean Restaurant,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,River,Road,Rock Club,Roof Deck,Rugby Pitch,Rugby Stadium,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Soccer Stadium,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [24]:
london_onehot.shape

(2472, 265)

#### Group rows by borough and by taking the mean of the frequency of occurrence of each category

In [25]:
london_grouped = london_onehot.groupby('Borough').mean().reset_index()
london_grouped

,Borough,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Bulgarian Restaurant,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Canal,Canal Lock,Caribbean Restaurant,Castle,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cricket Ground,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hotel,Hotel Bar,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Light Rail Station,Lighthouse,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,Nightclub,Noodle House,Okonomiyaki Restaurant,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Outlet Mall,Outlet Store,Palace,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Pub,Public Art,Racetrack,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Restaurant,River,Road,Rock Club,Roof Deck,Rugby Pitch,Rugby Stadium,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Scottish Restaurant,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Soccer Field,Soccer Stadium,Social Club,South Indian Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tiki Bar,Toy / Game Store,Track Stadium,Trail,Train Station,Tram Station,Tunnel,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Windmill,Wine Bar,Wine Shop,Winery,Women's Store,Yoga Studio
0,Barking and Dagenham,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.

#### Confirm the new size

In [26]:
london_grouped.shape

(32, 265)

#### Print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in london_grouped['Borough']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham----
           venue  freq
0    Gas Station  0.11
1    Supermarket  0.11
2       Bus Stop  0.11
3  Grocery Store  0.11
4      Racetrack  0.06


----Barnet----
                  venue  freq
0         Grocery Store  0.17
1                  Park  0.11
2                   Pub  0.11
3  Gym / Fitness Center  0.06
4                  Café  0.06


----Bexley----
                  venue  freq
0                   Pub  0.14
1        Clothing Store  0.07
2           Coffee Shop  0.07
3  Fast Food Restaurant  0.07
4    Italian Restaurant  0.05


----Brent----
                 venue  freq
0          Coffee Shop  0.10
1                Hotel  0.07
2       Clothing Store  0.07
3  Sporting Goods Shop  0.05
4    Indian Restaurant  0.05


----Bromley----
            venue  freq
0             Pub  0.13
1     Coffee Shop  0.08
2  Clothing Store  0.08
3     Pizza Place  0.05
4             Bar  0.05


----Camden----
            venue  freq
0     Coffee Shop  0.10
1            Café  0.0

#### Put that into a *pandas* dataframe
First, write a function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
boroughs_venues_sorted = pd.DataFrame(columns=columns)
boroughs_venues_sorted['Borough'] = london_grouped['Borough']

for ind in np.arange(london_grouped.shape[0]):
    boroughs_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

boroughs_venues_sorted.head()

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,Bus Stop,Gas Station,Supermarket,Grocery Store,Turkish Restaurant,Pub,Soccer Stadium,Park,Gym,Liquor Store
1,Barnet,Grocery Store,Pub,Park,Gym / Fitness Center,Dessert Shop,Dance Studio,Plaza,Cricket Ground,Pizza Place,Fish & Chips Shop
2,Bexley,Pub,Fast Food Restaurant,Coffee Shop,Clothing Store,Hotel,Supermarket,Grocery Store,Italian Restaurant,Museum,Bar
3,Brent,Coffee Shop,Clothing Store,Hotel,Sporting Goods Shop,Indian Restaurant,Bar,Grocery Store,Supermarket,Pizza Place,Italian Restaurant
4,Bromley,Pub,Coffee Shop,Clothing Store,Supermarket,Pizza Place,Bar,Electronics Store,Gym,Gym / Fitness Center,Park


## Cluster Boroughs

In [30]:
# set number of clusters
kclusters = 4

london_grouped_clustering = london_grouped.drop('Borough', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 2, 1, 2, 0, 2, 2, 2, 1], dtype=int32)

In [31]:
# add clustering labels
boroughs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = london_data

# merge london_grouped with london_data to add latitude/longitude for each borough
london_merged = london_merged.join(boroughs_venues_sorted.set_index('Borough'), on='Borough')

london_merged.dropna(inplace=True)

london_merged.head() # check the last columns!

,Borough,Latitude,Longitude,Crime count,Average house price(£),Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham,51.5607,0.1557,19836,312010.0,3,Bus Stop,Gas Station,Supermarket,Grocery Store,Turkish Restaurant,Pub,Soccer Stadium,Park,Gym,Liquor Store
1,Barnet,51.6252,-0.1517,30926,622784.0,3,Grocery Store,Pub,Park,Gym / Fitness Center,Dessert Shop,Dance Studio,Plaza,Cricket Ground,Pizza Place,Fish & Chips Shop
2,Bexley,51.4549,0.1505,16932,402823.0,2,Pub,Fast Food Restaurant,Coffee Shop,Clothing Store,Hotel,Supermarket,Grocery Store,Italian Restaurant,Museum,Bar
3,Brent,51.5588,-0.2817,33039,557527.0,1,Coffee Shop,Clothing Store,Hotel,Sporting Goods Shop,Indian Restaurant,Bar,Grocery Store,Supermarket,Pizza Place,Italian Restaurant
4,Bromley,51.4039,0.0198,25125,533716.0,2,Pub,Coffee Shop,Clothing Store,Supermarket,Pizza Place,Bar,Electronics Store,Gym,Gym / Fitness Center,Park


### Deliverable - 1. Visualize the resulting clusters

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Borough'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Cluster 1 - Commercial+Tourist Boroughs

In [33]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,51.5290,0,Coffee Shop,Café,Hotel,Pizza Place,Breakfast Spot,Bookstore,History Museum,Burger Joint,Exhibit,Grocery Store
18,51.5020,0,Pub,Hotel,Café,Restaurant,Italian Restaurant,Garden,English Restaurant,Park,Gym / Fitness Center,Bakery
23,51.5077,0,Hotel,Coffee Shop,Sandwich Place,Grocery Store,Restaurant,Supermarket,Pub,Park,Tapas Restaurant,Airport Service
26,51.5035,0,Coffee Shop,Hotel,Italian Restaurant,Bakery,Garden,Cocktail Bar,Scenic Lookout,Brewery,Pub,Tapas Restaurant
28,51.5099,0,Coffee Shop,Burger Joint,Hotel,Plaza,Park,Gym / Fitness Center,Bar,Lounge,Café,Music Venue
31,51.4973,0,Hotel,Park,Plaza,Coffee Shop,Theater,Bakery,Garden,Art Gallery,Café,Seafood Restaurant


### Cluster 2 - Suburban Boroughs with plenty of Shops

In [34]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,51.5588,1,Coffee Shop,Clothing Store,Hotel,Sporting Goods Shop,Indian Restaurant,Bar,Grocery Store,Supermarket,Pizza Place,Italian Restaurant
9,51.4892,1,Pub,Grocery Store,Coffee Shop,Fast Food Restaurant,Pier,Clothing Store,Platform,Plaza,Supermarket,Gym / Fitness Center
13,51.5898,1,Coffee Shop,Indian Restaurant,Sandwich Place,Clothing Store,Gym,Bar,Fast Food Restaurant,Afghan Restaurant,Pizza Place,Chocolate Shop
16,51.4746,1,Indian Restaurant,Hotel,Grocery Store,Coffee Shop,Clothing Store,Fast Food Restaurant,Sandwich Place,Asian Restaurant,Supermarket,Pub
24,51.5590,1,Grocery Store,Supermarket,Clothing Store,Coffee Shop,Fast Food Restaurant,Gym / Fitness Center,Sandwich Place,Bakery,Stationery Store,Theater


### Cluster 3 - Boroughs with lots of Pub/Coffee shops

In [35]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,51.4549,2,Pub,Fast Food Restaurant,Coffee Shop,Clothing Store,Hotel,Supermarket,Grocery Store,Italian Restaurant,Museum,Bar
4,51.4039,2,Pub,Coffee Shop,Clothing Store,Supermarket,Pizza Place,Bar,Electronics Store,Gym,Gym / Fitness Center,Park
6,51.3714,2,Pub,Coffee Shop,Clothing Store,Café,Hotel,Bookstore,Park,Indian Restaurant,Mediterranean Restaurant,Portuguese Restaurant
7,51.5130,2,Coffee Shop,Pub,Park,Hotel,Italian Restaurant,Pizza Place,Burger Joint,Indian Restaurant,Café,Sandwich Place
8,51.6538,2,Pub,Coffee Shop,Grocery Store,Pizza Place,Clothing Store,Fast Food Restaurant,Portuguese Restaurant,Bar,Bakery,Gym / Fitness Center
10,51.5450,2,Pub,Coffee Shop,Café,Bakery,Brewery,Pizza Place,Park,Bookstore,Vegetarian / Vegan Restaurant,Sandwich Place
11,51.4927,2,Pub,Café,Coffee Shop,Indian Restaurant,Italian Restaurant,Park,Gym / Fitness Center,Pizza Place,Japanese Restaurant,Plaza
12,51.6000,2,Pub,Coffee Shop,Café,Fast Food Restaurant,Grocery Store,Clothing Store,Bakery,Park,Mediterranean Restaurant,Sandwich Place
14,51.5812,2,Coffee Shop,Clothing Store,Pub,Supermarket,Fast Food Restaurant,Shopping Mall,Grocery Store,Park,Bar,Café
15,51.5441,2,Coffee Shop,Pub,Clothing Store,Italian Restaurant,Sandwich Place,Gym / Fitness Center,Hotel,Pizza Place,Burger Joint,Gym


### Cluster 4 - Quiet Boroughs

In [36]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,51.5607,3,Bus Stop,Gas Station,Supermarket,Grocery Store,Turkish Restaurant,Pub,Soccer Stadium,Park,Gym,Liquor Store
1,51.6252,3,Grocery Store,Pub,Park,Gym / Fitness Center,Dessert Shop,Dance Studio,Plaza,Cricket Ground,Pizza Place,Fish & Chips Shop


## Classify House prices and Crime counts

In [37]:
bin_labels = ['Very Cheap', 'Cheap', 'Average', 'Expensive', 'Very Expensive']
london_data['House Price Class'] = pd.qcut(london_data['Average house price(£)'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels)

bin_labels1 = ['Very Safe', 'Safe', 'Average', 'Dangerous', 'Very Dangerous']
london_data['Safety Class'] = pd.qcut(london_data['Crime count'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=bin_labels1)

london_data.head()

,Borough,Latitude,Longitude,Crime count,Average house price(£),House Price Class,Safety Class
0,Barking and Dagenham,51.5607,0.1557,19836,312010.0,Very Cheap,Very Safe
1,Barnet,51.6252,-0.1517,30926,622784.0,Expensive,Dangerous
2,Bexley,51.4549,0.1505,16932,402823.0,Very Cheap,Very Safe
3,Brent,51.5588,-0.2817,33039,557527.0,Average,Dangerous
4,Bromley,51.4039,0.0198,25125,533716.0,Average,Safe


### Deliverable - 2. Visualize house prices

In [38]:
# create map
map_house_price = folium.Map(location=[latitude, longitude], zoom_start=10)

def regioncolors(london_data):
    if london_data['House Price Class'] == 'Very Expensive':
        return 'purple'
    elif london_data['House Price Class'] == 'Expensive':
        return 'red'
    elif london_data['House Price Class'] == 'Average':
        return 'orange'
    elif london_data['House Price Class'] == 'Cheap':
        return 'green'
    else:
        return 'lightgreen'
    
london_data['House Price Class Colour'] = london_data.apply(regioncolors, axis=1)

# add markers to the map
for lat, lon, poi, pclass,classcolour in zip(london_data['Latitude'], london_data['Longitude'], london_data['Borough'], london_data['House Price Class'],london_data['House Price Class Colour']):
    label = folium.Popup(str(poi) + ', Price:' + str(pclass), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=classcolour,
        fill=True,
        fill_color=classcolour,
        fill_opacity=0.7).add_to(map_house_price)
       
map_house_price

### Deliverable - 3. Visualize borough safety

In [39]:
# create map
map_safety = folium.Map(location=[latitude, longitude], zoom_start=10)

def regioncolors1(london_data):
    if london_data['Safety Class'] == 'Very Dangerous':
        return 'purple'
    elif london_data['Safety Class'] == 'Dangerous':
        return 'red'
    elif london_data['Safety Class'] == 'Average':
        return 'orange'
    elif london_data['Safety Class'] == 'Safe':
        return 'green'
    else:
        return 'lightgreen'
    
london_data['Safety Class Colour'] = london_data.apply(regioncolors1, axis=1)

# add markers to the map
for lat, lon, poi, pclass,classcolour in zip(london_data['Latitude'], london_data['Longitude'], london_data['Borough'], london_data['Safety Class'],london_data['Safety Class Colour']):
    label = folium.Popup(str(poi) + ', Safety:' + str(pclass), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=classcolour,
        fill=True,
        fill_color=classcolour,
        fill_opacity=0.7).add_to(map_safety)
       
map_safety